In [5]:

#BOOKS

from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome('/Users/sammy/Documents/data/chromedriver')

def extract_record(item):
    #item
    atag = item.h2.a
    name = atag.text.strip()
    url = "https://www.amazon.com/" + atag.get('href')
    
    #price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
        
    #rating
    try:
        rating = item.i.text
    except AttributeError:
        rating = ''
    
        
    #image_url
    image = item.img.get("src")


    result = (name, price, rating, url, image)

    return result

import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def generate_url(search_term, page):
    base_template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ', '+')
    stem = base_template.format(search_term)
    url_template = stem + '&page={}'
    if page == 1:
        return stem
    else:
        return url_template.format(page)

def main(search):
    driver = webdriver.Chrome('/Users/sammy/Documents/data/chromedriver')
    records = []
    url = generate_url(search, 1)
    
    for page in range(1,8):
        search_url = generate_url(search, page)
        driver.get(search_url)
        
        html = driver.page_source
        soup = BeautifulSoup(html)
        results = soup.find_all("div", attrs={"class":"a-section a-spacing-medium"})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    with open('to_read_list.csv', 'w', newline='', encoding='utf-8')as f:
        writer = csv.writer(f)
        writer.writerow(['Name', 'Price', 'Rating', 'URL', 'Image'])
        writer.writerows(records)

main('self love')